# LINHA <> - Caderno de Operação

## Caderno de Inicialização e Calibração inicial da Linha de Luz

### Técnico responsável:

Texto inicial










## Notebook Initialization

The next cell loads the <b><i>util.ipynb</i></b> notebook, which provides functions and classes to connect the IJupyter and EPICS interfaces.

<sup><sup>Any bug or unexpected behavior from any of these functions and classes, please contact SOL.</sup></sup>

In [1]:
import io, os, sys, types
from IPython import get_ipython
import nbformat
from IPython.core.interactiveshell import InteractiveShell


def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, nbformat.NO_CONVERT)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [2]:
from util import *

importing Jupyter notebook from util.ipynb


## Configuration Cell

Check the boxes to set some notebook's configuration

In [3]:
checkbox_logprint_in_cell = widgets.Checkbox(
    value=False,
    description="Print log in Notebook's cells",
    disabled=False,
    style={'description_width': 'initial'},
)

config = {"log_cell": checkbox_logprint_in_cell}

output = widgets.Output()

@output.capture()
def change_checkbox_logprint_in_cell(change):
    logprint("Changed Logprint Checkbox to: " + str(change.new))
    
checkbox_logprint_in_cell.observe(change_checkbox_logprint_in_cell, 'value')

box = widgets.VBox([checkbox_logprint_in_cell, output])

display(box)

plots_list = []


## Motor Calibration

### Write in the Text Box, separated by spaces, every motor name that should be initialized.

###### PS: Do not use any other character separation other than spaces!

In [4]:
config

{'log_cell': Checkbox(value=False, description="Print log in Notebook's cells", style=DescriptionStyle(description_width='initial'))}

In [5]:
start_motor_button_box = StartMotorsButton(config)
start_motor_button_box.display_start_button()

Output()

### Write in the "Number Text Box" the absolute target value desired for the motor 

In [6]:
start_motor_button_box.display_motors_targ_buttons()

### Write in the Text Box, separated by spaces, every motor PV name that should be monitored.

###### PS: Do not use any other character separation other than spaces!

In [7]:
motor_monitor = MotorsMonitor(config)
motor_monitor.display_monitor_motors()

Output()

In [8]:
# from py4syn.utils.scan import scan
# scan(cnew_motor, 10, 5, 10, 0.1)

In [9]:
pv_monitor = PVMonitor(config)
pv_monitor.display_monitor_pvs()

Output()

## Call Scan

## Plotting data

### Plotly

Plotly is a great library for plotting data in a Jupyter Notebook.

Informations and examples of how to use this library can be found in https://plot.ly/python/


#### Simple scatter plot example:

In [27]:
import json
import os
import threading
from IPython.display import display
import ipywidgets as widgets
import time
import pandas
import subprocess


class MonitorScanSave(widgets.Button):
    
    def __init__(self, config, plots_list, *args, **kwargs):
        widgets.Button.__init__(self, *args, **kwargs)
        
        # Config
        self.config = config
        
        # class Button values for MonitorScanSave
        self.description='Start Scanning Plot'
        self.disabled=False
        self.button_style='success'
        self.tooltip='Click me'
        self.icon=''
        self.layout = widgets.Layout(width='300px')
        
        # Scan save file and directory
        self.scan_save_dir = '/tmp/'
        self.scan_save_file = 'scan_gui.temp'

        self.scan_path = Path(self.scan_save_dir + self.scan_save_file)
        
        # Logging
        self.output = widgets.Output()
        
        # Threading
        self.monitor = False
        self.thread = None
        self.refresh_thread = None
        
        # Set callback function for click event
        self.on_click(self._start_button)
        
        # Widgets displays
        self.start_button = widgets.VBox([self])
        
        # Clean previous temp config file
        try:
            os.remove(str(self.scan_path))
        except:
            pass
        
        self.checkbox_live_plot = widgets.Checkbox(
            value=False,
            description="Live plot in Jupyter: ",
            disabled=False,
            style={'description_width': 'initial'},
        )
        
        self.checkbox_final_plot = widgets.Checkbox(
            value=False,
            description="Plot scan-gui graph instead of Jupyter's: ",
            disabled=False,
            style={'description_width': 'initial'},
        )
        
        
        
        self.fig = go.FigureWidget()
        self.fig_box = widgets.Box()
        self.refresh_icon_box = widgets.Box(layout=widgets.Layout(width='40px', height='40px'))
        
        self.plots_list = plots_list
        
        self.export = False
        self.clear_threads = False
        
    @staticmethod
    def _start_button(b):
        # Clear previous logs outputs
        b.output.clear_output()
        
        # with statement to output logs in stdou (if this option is enabled)
        with b.output:
            if b.monitor:
                # Enable checkboxes
                b.checkbox_live_plot.disabled = False
                b.checkbox_final_plot.disabled = False
                
                # Change button monitor status
                b.monitor = not b.monitor
                
                # Change button to a "clicked status"
                b.disabled = True
                b.button_style = ''
                b.description='Stopping...'
                
                # We should sleep for some time to give some responsiveness to the user
                time.sleep(0.5)

                # Stop thread to monitor the save file
                try:
                    logprint("Stopping threads", config=b.config)
                    b.thread.join()
                    b.fig_thread.join()
                    b.refresh_thread.join()
                    
                    b.clear_threads = False
                except Exception as e:
                    # If any error occurs, log that but dont stop code exection
                    logprint("Error in stopping threads", "[ERROR]", config=b.config)
                    logprint(str(e), "[ERROR]", config=b.config)

                # Change button layout monitoring
                b.disabled = False
                b.button_style = 'success'
                b.description='Start Scanning Plot'
            else:
                # Disable checkboxes
                b.checkbox_live_plot.disabled = True
                b.checkbox_final_plot.disabled = True
                
                p = subprocess.Popen(["pydm --hide-nav-bar --hide-menu-bar /home/gabriel.andrade/work/scan-gui/scan_gui.py"],
                                     shell=True)
                
                # Change button monitor status
                b.monitor = not b.monitor
                
                # Change button to a "clicked status"
                b.disabled = True
                b.button_style = ''
                b.description='Starting...'
                
                # We should sleep for some time to give some responsiveness to the user
                time.sleep(0.5)
                
                # Clean previous scans config
                try:
                    os.remove(str(b.scan_path))
                except:
                    pass

                # Start thread to monitor the save file
                try:
                    logprint("Starting thread", config=b.config)
                    b.thread = threading.Thread(target=b.monitor_save_file)
                    b.thread.start()
                except Exception as e:
                    # If any error occurs, log that but dont stop code exection
                    logprint("Error in starting thread", "[ERROR]", config=b.config)
                    logprint(str(e), "[ERROR]", config=b.config)

                # Change button layout monitoring
                b.disabled = False
                b.button_style = 'danger'
                b.description='Stop Scanning Plot'
            
    
    def monitor_save_file(self):
        with self.output:
            while self.monitor:
                if self.scan_path.is_file():
                    # Started scan
                    self.started_scan = True
                    
                    with open(str(self.scan_path)) as file:
                        try: 
                             save_file = json.load(file)
                        except ValueError: 
                             pass
                    
                    os.remove(str(self.scan_path))
                    
                    save_file_str = json.dumps(save_file)
                    save_file_pretty_str = json.dumps(save_file, indent=4)
                    
                    command = save_file["command"]["value"]
                    parser = self.scan_parser()
                    
                    self.synchronous = save_file["checkSync"]["value"]                    
                    self.scan_name = self.get_scan_name(command, parser)
                    config_name = self.get_config_name(command, parser)
                    
                    self.plot_name = self.scan_name + "-jupy.png"
                    
                    ts = time.gmtime()
    
                    year_month_day = time.strftime("%Y-%m-%d", ts)
                    time_stamp = time.strftime("%Y-%m-%d %H:%M:%S", ts) + " UTC-0"
                    self.log_str = time_stamp + "| [SCAN]:\n" + \
                              "Scan with command: '" + command + "'\n" + \
                              "Scan configuration: '" + config_name + "'\n" + \
                              "Scan data saved in: '" + self.scan_name + "'\n" + \
                              "Scan plot saved in: '" + self.plot_name + "'\n"
                    
                    log_file_name = Path('./' + year_month_day + '-scanlog.txt')
                    with open(str(log_file_name), "a") as f:
                        f.write(self.log_str + '\n')
                    IPython.display.update_display(IPython.display.Pretty(self.log_str), display_id='text')
                                    
                    self.plots_list.append([self, self.plot_name])

                    # Call live graph
                    self.list_motors = save_file["listMotors"]["value"]

                    self.fig_thread = threading.Thread(target=self.thread_plot)
                    self.fig_thread.start()
                    
                    # Scan status icon
                    self.refresh_thread = threading.Thread(target=self.thread_refresh_icon)
                    self.refresh_thread.start()  
                else:
                    pass
                
                if self.clear_threads:
                    self.fig_thread.join()
                    self.refresh_thread.join()
                    
                    self.clear_threads = False
                
                time.sleep(0.5)
                
    def get_scan_name(self, command, parser):
        args = parser.parse_known_args(command.split(' '))

        fileName = args[0].output

        leadingZeros = 4
        newName = ""
        cont = 0
        while(True):
            cont += 1
            newName = fileName + "_" + str(cont).zfill(leadingZeros)
            if(os.path.isfile(newName)):
                continue
            else:
#                 if self.synchronous:
#                     newName = fileName + "_" + str(cont - 1).zfill(leadingZeros)
                break
                
        return newName
    
    def get_config_name(self, command, parser):
        args = parser.parse_known_args(command.split(' '))

        config_name = args[0].configuration
        
        return config_name
    
    def scan_parser(self):
        parser = ScanParser()
        
        return parser.parser
    
    def update_pd(self, default_name, label):
        try:
            df = pd.read_csv(default_name, sep=' ', comment='#', header=None)
        except Exception as e:
            return pd.DataFrame(), label

        filtered_label = label
        if not label:
            labels = []
            with open(default_name) as file:
                for i, line in enumerate(file):
                    if i == 6:
                        self.number_reads = (int(line.split(' ')[1]))
                    elif i == 8:
                        labels = line.split(' ')[1:]
                        break

            labels = list(filter(lambda x: x != '', labels))

            for item in labels:
                filtered_label.append(item.rstrip('\n'))

            label = filtered_label

        df.columns = pd.Index(filtered_label, dtype='object')
        return df, label

    def thread_plot(self):
        df = pd.DataFrame()
        while df.empty:
            label = []
            df, label = self.update_pd(self.scan_name, label)
        
        number_motors = len(self.list_motors)
        self.create_figure(len(df.columns) - number_motors)
        
        while df.shape[0] < self.number_reads:
            df, label = self.update_pd(self.scan_name, label)
            if self.checkbox_live_plot.value:           
                if df.empty:
                    continue

                for i in range(len(df.columns) - number_motors): 
                    self.fig['data'][i]['x'] = df.index.values
                    self.fig['data'][i]['y'] = df[df.columns[number_motors + i]].values               

            time.sleep(1)
        
        # Finished scan
        self.started_scan = False
        self.clear_threads = True
        
        # update last scan value
        for i in range(len(df.columns) - number_motors): 
                self.fig['data'][i]['x'] = df.index.values
                self.fig['data'][i]['y'] = df[df.columns[number_motors + i]].values
                
        # save image as png
        pio.write_image(self.fig, self.plot_name)
        
        # Plot scan-gui pyqt graph
        if self.checkbox_final_plot.value:
            self.load_image_file(self.scan_name + ".png")
            
#         self._start_button(self)
            
    def create_figure(self, number_traces):
        self.traces = []
        
        self.fig = go.FigureWidget(tools.make_subplots(rows=number_traces, cols=1))
        
        for i in range(number_traces):
            trace = go.Scatter(
                x=[], y=[], # Data
                mode='lines+markers', name='line' + str(i+1)
            )

            self.traces.append(trace)
            self.fig.append_trace(trace, i + 1, 1) # using i + 1 because plot index starts at 1

        self.fig['layout'].update(title='Scan', plot_bgcolor='rgb(230, 230, 230)')
        self.fig_box.children = (self.fig,)
        
    def thread_refresh_icon(self):
        self.refresh_icon_box.layout = widgets.Layout(width='40px', height='40px')
        
        file = open(".img/refresh_00.png", "rb")
        image = file.read()
        img_w = widgets.Image(
            value=image,
            format='png',
            width=35,
            height=35,
        )
        
        self.refresh_icon_box.children = (img_w,)

        i = 0
        f = 0
        while self.started_scan:
            time.sleep(0.1)
            i += 1
            file = open(".img/refresh_" + str(i).zfill(2) + ".png", "rb")
            image = file.read()
            img_w .value = image
            if i == 11:
                i = -1
            f += 1
        
        # Ended scan, blank the resfresh image
        file = open(".img/tick.png", "rb")
        image = file.read()
        img_w .value = image
        
    def clean_refresh_icon(self):
        file = open(".img/blank.png", "rb")
        image = file.read()
        img_w = widgets.Image(
            value=image,
            format='png',
            width=35,
            height=35,
        )
        
        self.refresh_icon_box.children = (img_w,)
        
    def export_image_thread(self):
        updating = True
        while True:
            if self.export:
                if updating:
                    img = IPython.display.Image(data=self.plot_name)
                    IPython.display.update_display(img, display_id='img')
                    updating = False
                    
            else:
                if not updating:
                    IPython.display.update_display("", display_id='img')
                    updating = True
                
            time.sleep(0.5)
            
    def load_image_file(self, filename):
        self.fig_box.children = []
        
        img = IPython.display.Image(data=filename)
        IPython.display.update_display(img, display_id='img')
    
    def display_start_button(self):
        display(self.checkbox_live_plot, self.checkbox_final_plot, self.start_button, self.refresh_icon_box, self.fig_box, self.output)
        IPython.display.display((""), display_id='text')
        IPython.display.display((""), display_id='img')
        self.export_thread = threading.Thread(target=self.export_image_thread)
        self.export_thread.start()


In [26]:
import ipywidgets as widgets
import subprocess


class ManualAlignmentButton(widgets.Button):
    
    def __init__(self, config, *args, **kwargs):
        widgets.Button.__init__(self, *args, **kwargs)
        
        # Config
        self.config = config
        
        # class Button values for MonitorScanSave
        self.description = 'Start Manual Alignment'
        self.disabled = False
        self.button_style = 'success'
        self.tooltip = 'Click me'
        self.icon = ''
        self.layout = widgets.Layout(width='300px')
               
        # Logging
        self.output = widgets.Output()
        
        # Set callback function for click event
        self.on_click(self._start_button)
        
        # Widgets displays
        self.start_button = widgets.VBox([self])
        
    @staticmethod
    def _start_button(b):
        # Clear previous logs outputs
        b.output.clear_output()
        
        # with statement to output logs in stdou (if this option is enabled)
        with b.output:
            # Change button to a "clicked status"
            b.disabled = True
            b.button_style = ''
            b.description='Aligning...'

            # We should sleep for some time to give some responsiveness to the user
            time.sleep(0.5)

            # Stop thread to monitor the save file
            try:
                logprint("Starting manual alignment", config=b.config)
                p = subprocess.run(["pydm /home/gabriel.andrade/work/sol-widgets/examples/motor/slits.ui"],
                                 shell=True, check=True)

                logprint("Finished manual alignment", config=b.config)
            except Exception as e:
                # If any error occurs, log that but dont stop code exection
                logprint("Error in manual alignment", "[ERROR]", config=b.config)
                logprint(str(e), "[ERROR]", config=b.config)

            # Change button layout monitoring
            b.disabled = False
            b.button_style = 'success'
            b.description='Start Manual Alignment'
    
    def display_start_button(self):
        display(self.start_button, self.output)


In [25]:
a = ManualAlignmentButton(config)
a.display_start_button()

Output()

In [11]:
%tb
live_plot = MonitorScanSave(config, plots_list)
live_plot.display_start_button()

No traceback available to show.


Checkbox(value=False, description='Live plot in Jupyter: ', style=DescriptionStyle(description_width='initial'…

Checkbox(value=False, description="Plot scan-gui graph instead of Jupyter's: ", style=DescriptionStyle(descrip…

Box(layout=Layout(height='40px', width='40px'))

Box()

Output()

2019-05-13 13:20:36 UTC-0| [SCAN]:
Scan with command: 'scan -c default -o /home/gabriel.andrade/work/jupy4syn/scans/plot -s --snake --motor solm1 --start 1.0 --end 5.0 --step-or-points 1.0 --time 0.1'
Scan configuration: 'default'
Scan data saved in: '/home/gabriel.andrade/work/jupy4syn/scans/plot_0234'
Scan plot saved in: '/home/gabriel.andrade/work/jupy4syn/scans/plot_0234.png'


''

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [31]:
class ExportButtonPDF(widgets.Button):
    def __init__(self, config, plots_list, *args, **kwargs):
        widgets.Button.__init__(self, *args, **kwargs)
        
        # Config
        self.config = config
        self.description='Export Notebook to PDF'
        self.disabled=False
        self.button_style='warning' # 'success', 'info', 'warning', 'danger' or ''
        self.tooltip='Click me'
        self.icon=''
        self.layout = widgets.Layout(width='300px')
    
        self.on_click(self._export_button)
        
        self.plots_list = plots_list
    
    @staticmethod
    def _export_button(b):
        # Change button to a "clicked status"
        b.disabled = True
        b.button_style = ''
        b.description='Exporting...'

        # We should sleep for some time to give some responsiveness to the user
        time.sleep(0.5)
        
        try:
            from IPython.display import Javascript
    
            year_month_day = time.strftime("%Y-%m-%d", ts)
            time_stamp = time.strftime("%Y-%m-%d-%H:%M:%S", ts)
            output_file = time_stamp + "-main_notebook"

            display(Javascript('IPython.notebook.save_checkpoint();'))
            
            time.sleep(3)
            
            os.system("python3 -m nbconvert main_notebook.ipynb --output-dir=./exports --output=" + output_file + " --to pdf")
            
            for plot in b.plots_list:
                plot[0].export = False
        except:
            pass
        
        # Reenable button
        b.disabled = False
        b.button_style = 'warning'
        b.description='Export Notebook to PDF'
        
    def display_export_button(self):
        display(self)
                    
but = ExportButtonPDF(config, plots_list)
but.display_export_button()

ExportButtonPDF(button_style='warning', description='Export Notebook to PDF', layout=Layout(width='300px'), st…

In [32]:
class ExportButtonHTML(widgets.Button):
    def __init__(self, config, plots_list, *args, **kwargs):
        widgets.Button.__init__(self, *args, **kwargs)
        
        # Config
        self.config = config
        self.description='Export Notebook to HTML'
        self.disabled=False
        self.button_style='warning' # 'success', 'info', 'warning', 'danger' or ''
        self.tooltip='Click me'
        self.icon=''
        self.layout = widgets.Layout(width='300px')
    
        self.on_click(self._export_button)
        
        self.plots_list = plots_list
    
    @staticmethod
    def _export_button(b):
        # Change button to a "clicked status"
        b.disabled = True
        b.button_style = ''
        b.description='Exporting...'

        # We should sleep for some time to give some responsiveness to the user
        time.sleep(0.5)
        
        try:
            from IPython.display import Javascript
    
            year_month_day = time.strftime("%Y-%m-%d", ts)
            time_stamp = time.strftime("%Y-%m-%d-%H:%M:%S", ts)
            output_file = time_stamp + "-main_notebook"

            display(Javascript('IPython.notebook.save_checkpoint();'))
            
            time.sleep(3)
            
            os.system("python3 -m nbconvert main_notebook.ipynb --output-dir=./exports --output=" + output_file + " --to html")
            
            for plot in b.plots_list:
                plot[0].export = False
        except:
            pass
        
        # Reenable button
        b.disabled = False
        b.button_style = 'warning'
        b.description='Export Notebook to HTML'
        
    def display_export_button(self):
        display(self)
                    
but = ExportButtonHTML(config, plots_list)
but.display_export_button()

ExportButtonHTML(button_style='warning', description='Export Notebook to HTML', layout=Layout(width='300px'), …